In [1]:
import pandas as pd
import numpy as np
import datetime
import glob

In [9]:
symbol = 'CAT'

filename = glob.glob("../raw_data/{}*.csv".format(symbol))[0].split('/')[2]
print(filename)

CAT.USUSD_Candlestick_1_M_BID_11.05.2017-15.02.2020.csv


In [8]:
data = pd.read_csv('../raw_data/'+filename)
print('Size: ', data.shape[0], ' datapoints')

data = data.rename(columns={'Local time' : 'Date Time'})
data['Date Time'] = data['Date Time'].apply(lambda d: datetime.datetime.strptime(d[:-14], '%d.%m.%Y %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S'))
data = data.set_index('Date Time')

print(data.head())
print(data.tail())

Size:  1455900  datapoints
                       Open    High     Low   Close  Volume
Date Time                                                  
2017-05-11 00:00:00  101.76  101.76  101.76  101.76     0.0
2017-05-11 00:01:00  101.76  101.76  101.76  101.76     0.0
2017-05-11 00:02:00  101.76  101.76  101.76  101.76     0.0
2017-05-11 00:03:00  101.76  101.76  101.76  101.76     0.0
2017-05-11 00:04:00  101.76  101.76  101.76  101.76     0.0
                        Open     High      Low    Close  Volume
Date Time                                                      
2020-02-15 23:55:00  137.978  137.978  137.978  137.978     0.0
2020-02-15 23:56:00  137.978  137.978  137.978  137.978     0.0
2020-02-15 23:57:00  137.978  137.978  137.978  137.978     0.0
2020-02-15 23:58:00  137.978  137.978  137.978  137.978     0.0
2020-02-15 23:59:00  137.978  137.978  137.978  137.978     0.0


In [10]:
def isTrading(data):
    truth = np.array([dp.weekday() not in [5,6] for dp in data])
    truth = np.vstack((truth, data.hour < 14))
    truth = np.vstack((truth, np.vstack((data.hour > 7, np.vstack((data.hour == 7, data.minute >= 30)).all(axis=0))).any(axis=0)))
                                 
    return truth.all(axis=0)

dateTimes = pd.to_datetime(data.index)
tradingIdxs = isTrading(dateTimes)

In [11]:
tradingData = data[tradingIdxs]
print(tradingData.head())
print(tradingData.tail())

                       Open    High     Low   Close  Volume
Date Time                                                  
2017-05-11 07:30:00  101.76  101.76  101.29  101.29    0.15
2017-05-11 07:31:00  101.10  101.68  101.05  101.68    0.19
2017-05-11 07:32:00  101.45  101.60  101.45  101.51    0.10
2017-05-11 07:33:00  101.53  101.62  101.46  101.48    0.09
2017-05-11 07:34:00  101.53  101.55  101.53  101.54    0.16
                        Open     High      Low    Close  Volume
Date Time                                                      
2020-02-14 13:55:00  137.918  137.938  137.857  137.888  0.0930
2020-02-14 13:56:00  137.898  138.038  137.888  138.038  0.0765
2020-02-14 13:57:00  138.037  138.038  137.957  137.958  0.1258
2020-02-14 13:58:00  137.938  138.008  137.937  137.987  0.1023
2020-02-14 13:59:00  137.988  137.998  137.888  137.978  0.2017


In [12]:
tradingData.to_csv('../clean_data/'+filename)